# Balatro-ish. An homage to Balatro
https://claude.site/artifacts/bfe5a179-655d-4cca-bd2d-d32e53daa2ba

# Key Features

Standard 52-card deck:
* The game initializes with a complete deck of cards
* Blind selection: Start by choosing a blind (or skip) which affects your starting ante and jokers
* 8-card hands: Draws 8 cards for each hand from your deck
* 4 hands limit: You can only play 4 hands per game
* 4 discards limit: You're limited to 4 discards throughout the game
* Sorting functionality: Cards can be sorted by rank or suit with the toggle button
* Hand evaluation: Evaluates poker hands (pairs, three of a kind, straight, flush, etc.)
* Scoring system: Each hand type has a base score that gets multiplied by your jokers
* Progression system: Ante increases every 2 rounds, and you gain new jokers every 3 rounds

# How to Play

* Start by selecting a blind (or skipping) to begin the game
* Click cards you want to discard (they'll highlight blue)
* Click "Discard Selected" to replace those cards with new ones from the deck
* When you're happy with your hand, click "Play Hand" to score it
* The game continues until you've played all 4 hands
* Your final score is the total chips you've collected

Enjoy! This is a simplified version, but it captures the core loop of Balatro. Please check out the full game here: https://www.balatro.co/

In [ ]:
# Balatro-ish - a browser-based game attempt.

import { useState, useEffect } from 'react';
import { ArrowUpDown } from 'lucide-react';

export default function Balatro() {
  // Game states
  const [gamePhase, setGamePhase] = useState('blindSelection'); // 'blindSelection', 'playing', 'gameOver'
  const [deck, setDeck] = useState([]);
  const [hand, setHand] = useState([]);
  const [discardCount, setDiscardCount] = useState(4);
  const [handsLeft, setHandsLeft] = useState(4);
  const [selectedCards, setSelectedCards] = useState([]);
  const [chips, setChips] = useState(0);
  const [ante, setAnte] = useState(1);
  const [round, setRound] = useState(1);
  const [jokers, setJokers] = useState([]);
  const [score, setScore] = useState(0);
  const [finalScore, setFinalScore] = useState(0);
  const [sortType, setSortType] = useState('rank'); // 'rank' or 'suit'

  // Initialize a new game
  useEffect(() => {
    initializeDeck();
  }, []);

  // Sort cards whenever sort type changes or hand changes
  useEffect(() => {
    if (hand.length > 0) {
      sortHand();
    }
  }, [sortType, hand.length]);

  // Initialize a standard 52-card deck
  const initializeDeck = () => {
    const suits = ['hearts', 'diamonds', 'clubs', 'spades'];
    const ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A'];
    const newDeck = [];

    for (const suit of suits) {
      for (const rank of ranks) {
        newDeck.push({ suit, rank, selected: false });
      }
    }

    // Shuffle the deck
    for (let i = newDeck.length - 1; i > 0; i--) {
      const j = Math.floor(Math.random() * (i + 1));
      [newDeck[i], newDeck[j]] = [newDeck[j], newDeck[i]];
    }

    setDeck(newDeck);
  };

  // Select a blind and start the game
  const selectBlind = (blindType) => {
    let startingAnte = 1;
    let startingJokers = [];

    switch (blindType) {
      case 'small':
        startingAnte = 2;
        startingJokers = [{ name: 'Basic Joker', multiplier: 2 }];
        break;
      case 'medium':
        startingAnte = 3;
        startingJokers = [
          { name: 'Basic Joker', multiplier: 2 },
          { name: 'Lucky Joker', multiplier: 1.5 }
        ];
        break;
      case 'big':
        startingAnte = 5;
        startingJokers = [
          { name: 'Basic Joker', multiplier: 2 },
          { name: 'Lucky Joker', multiplier: 1.5 },
          { name: 'Steel Joker', multiplier: 3 }
        ];
        break;
      default: // Skip
        startingJokers = [{ name: 'Basic Joker', multiplier: 1.5 }];
    }

    setAnte(startingAnte);
    setJokers(startingJokers);
    drawInitialHand();
    setGamePhase('playing');
  };

  // Draw initial 8 cards
  const drawInitialHand = () => {
    const newHand = deck.slice(0, 8);
    const remainingDeck = deck.slice(8);
    setHand(newHand);
    setDeck(remainingDeck);
  };

  // Toggle card selection
  const toggleCardSelection = (index) => {
    if (discardCount > 0) {
      const updatedHand = [...hand];
      updatedHand[index].selected = !updatedHand[index].selected;
      setHand(updatedHand);

      if (updatedHand[index].selected) {
        setSelectedCards([...selectedCards, index]);
      } else {
        setSelectedCards(selectedCards.filter(i => i !== index));
      }
    }
  };

  // Discard selected cards and draw new ones
  const discardSelected = () => {
    if (selectedCards.length === 0 || discardCount <= 0) return;

    // Create a new hand by replacing selected cards
    const newHand = [...hand];
    const newDeck = [...deck];

    // Sort indices in descending order to avoid offset issues when replacing
    const sortedIndices = [...selectedCards].sort((a, b) => b - a);

    // Replace each selected card with a new one from the deck
    for (const index of sortedIndices) {
      if (newDeck.length > 0) {
        newHand[index] = newDeck.pop();
        newHand[index].selected = false;
      }
    }

    setHand(newHand);
    setDeck(newDeck);
    setSelectedCards([]);
    setDiscardCount(discardCount - 1);

    // Sort the hand after drawing new cards
    sortHand(newHand);
  };

  // Sort the hand based on current sort type
  const sortHand = (cardsToSort = [...hand]) => {
    const rankOrder = {'2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '10': 10, 'J': 11, 'Q': 12, 'K': 13, 'A': 14};
    const suitOrder = {'hearts': 0, 'diamonds': 1, 'clubs': 2, 'spades': 3};

    let sortedHand = [...cardsToSort];

    if (sortType === 'rank') {
      sortedHand.sort((a, b) => {
        // First sort by rank (descending)
        if (rankOrder[b.rank] !== rankOrder[a.rank]) {
          return rankOrder[b.rank] - rankOrder[a.rank];
        }
        // Then by suit
        return suitOrder[a.suit] - suitOrder[b.suit];
      });
    } else {
      sortedHand.sort((a, b) => {
        // First sort by suit
        if (suitOrder[a.suit] !== suitOrder[b.suit]) {
          return suitOrder[a.suit] - suitOrder[b.suit];
        }
        // Then by rank (descending)
        return rankOrder[b.rank] - rankOrder[a.rank];
      });
    }

    setHand(sortedHand);
  };

  // Toggle sort type
  const toggleSortType = () => {
    setSortType(sortType === 'rank' ? 'suit' : 'rank');
  };

  // Evaluate the poker hand and calculate score
  const evaluateHand = () => {
    // Count occurrences of each rank and suit
    const rankCounts = {};
    const suitCounts = {};
    hand.forEach(card => {
      rankCounts[card.rank] = (rankCounts[card.rank] || 0) + 1;
      suitCounts[card.suit] = (suitCounts[card.suit] || 0) + 1;
    });

    // Check for different poker hands (simplified)
    let handType = "";
    let baseScore = 0;

    // Check for flush
    const isFlush = Object.values(suitCounts).some(count => count >= 5);

    // Check for straight (simplified)
    const ranks = hand.map(card => card.rank);
    const isStreet = (
      ranks.includes('A') &&
      ranks.includes('K') &&
      ranks.includes('Q') &&
      ranks.includes('J') &&
      ranks.includes('10')
    ) || (
      ranks.includes('K') &&
      ranks.includes('Q') &&
      ranks.includes('J') &&
      ranks.includes('10') &&
      ranks.includes('9')
    );

    // Find pairs, three of a kind, etc.
    const pairs = Object.values(rankCounts).filter(count => count === 2).length;
    const threeOfAKind = Object.values(rankCounts).filter(count => count === 3).length;
    const fourOfAKind = Object.values(rankCounts).filter(count => count === 4).length;

    // Determine hand type and base score
    if (isFlush && isStreet) {
      handType = "Straight Flush";
      baseScore = 100;
    } else if (fourOfAKind) {
      handType = "Four of a Kind";
      baseScore = 80;
    } else if (threeOfAKind && pairs) {
      handType = "Full House";
      baseScore = 70;
    } else if (isFlush) {
      handType = "Flush";
      baseScore = 60;
    } else if (isStreet) {
      handType = "Straight";
      baseScore = 50;
    } else if (threeOfAKind) {
      handType = "Three of a Kind";
      baseScore = 40;
    } else if (pairs >= 2) {
      handType = "Two Pair";
      baseScore = 30;
    } else if (pairs === 1) {
      handType = "Pair";
      baseScore = 20;
    } else {
      handType = "High Card";
      baseScore = 10;
    }

    // Apply joker multipliers
    let totalMultiplier = 1;
    jokers.forEach(joker => {
      totalMultiplier *= joker.multiplier;
    });

    const finalScore = Math.floor(baseScore * totalMultiplier);
    return { handType, baseScore, finalScore };
  };

  // Play the current hand
  const playHand = () => {
    if (handsLeft <= 0) return;

    const { handType, baseScore, finalScore } = evaluateHand();
    setScore(finalScore);

    // Add score to chips
    const roundWinnings = finalScore * ante;
    setChips(chips + roundWinnings);

    // Add a new joker every 3 rounds
    if (round % 3 === 0) {
      const newJoker = {
        name: `Round ${round} Joker`,
        multiplier: 1 + (Math.random() * 2).toFixed(1)
      };
      setJokers([...jokers, newJoker]);
    }

    // Increase ante every 2 rounds
    if (round % 2 === 0) {
      setAnte(ante + 1);
    }

    // Draw a new hand
    if (handsLeft > 1) {
      // Make sure we have enough cards
      if (deck.length < 8) {
        initializeDeck();
      }

      const newHand = deck.slice(0, 8);
      const remainingDeck = deck.slice(8);
      setHand(newHand);
      setDeck(remainingDeck);
      sortHand(newHand);
    } else {
      // Game over
      setFinalScore(chips);
      setGamePhase('gameOver');
    }

    setRound(round + 1);
    setHandsLeft(handsLeft - 1);
  };

  // Start a new game
  const startNewGame = () => {
    initializeDeck();
    setHand([]);
    setDiscardCount(4);
    setHandsLeft(4);
    setSelectedCards([]);
    setChips(0);
    setAnte(1);
    setRound(1);
    setJokers([]);
    setScore(0);
    setFinalScore(0);
    setGamePhase('blindSelection');
  };

  // Render card with suit symbol and color
  const renderCard = (card, index) => {
    const suitSymbols = {
      'hearts': '♥',
      'diamonds': '♦',
      'clubs': '♣',
      'spades': '♠'
    };

    const suitColors = {
      'hearts': 'text-red-600',
      'diamonds': 'text-red-600',
      'clubs': 'text-black',
      'spades': 'text-black'
    };

    return (
      <div
        key={index}
        className={`w-16 h-24 bg-white border-2 rounded-lg p-2 flex flex-col justify-between cursor-pointer ${card.selected ? 'border-blue-500 bg-blue-100' : 'border-gray-300'}`}
        onClick={() => toggleCardSelection(index)}
      >
        <div className="text-left font-bold">
          <span className={suitColors[card.suit]}>{card.rank}</span>
        </div>
        <div className={`text-center text-2xl ${suitColors[card.suit]}`}>
          {suitSymbols[card.suit]}
        </div>
        <div className="text-right font-bold">
          <span className={suitColors[card.suit]}>{card.rank}</span>
        </div>
      </div>
    );
  };

  // Render blind selection screen
  if (gamePhase === 'blindSelection') {
    return (
      <div className="p-6 max-w-2xl mx-auto bg-gray-100 rounded-lg">
        <h1 className="text-2xl font-bold mb-4 text-center">Balatro-Inspired Game</h1>
        <div className="mb-6 text-center">
          <h2 className="text-xl mb-2">Select a Blind</h2>
          <p className="mb-4">Choose your starting condition:</p>
          <div className="flex flex-wrap justify-center gap-4">
            <button
              className="p-4 bg-blue-500 text-white rounded hover:bg-blue-600"
              onClick={() => selectBlind('small')}
            >
              Small Blind<br/>
              (Ante: 2, 1 Joker)
            </button>
            <button
              className="p-4 bg-green-500 text-white rounded hover:bg-green-600"
              onClick={() => selectBlind('medium')}
            >
              Medium Blind<br/>
              (Ante: 3, 2 Jokers)
            </button>
            <button
              className="p-4 bg-red-500 text-white rounded hover:bg-red-600"
              onClick={() => selectBlind('big')}
            >
              Big Blind<br/>
              (Ante: 5, 3 Jokers)
            </button>
            <button
              className="p-4 bg-gray-500 text-white rounded hover:bg-gray-600"
              onClick={() => selectBlind('skip')}
            >
              Skip Blind<br/>
              (Ante: 1, Basic Joker)
            </button>
          </div>
        </div>
      </div>
    );
  }

  // Render game over screen
  if (gamePhase === 'gameOver') {
    return (
      <div className="p-6 max-w-2xl mx-auto bg-gray-100 rounded-lg">
        <h1 className="text-2xl font-bold mb-4 text-center">Game Over</h1>
        <div className="mb-6 text-center">
          <h2 className="text-xl mb-4">Final Score: {finalScore} chips</h2>
          <button
            className="px-4 py-2 bg-blue-500 text-white rounded hover:bg-blue-600"
            onClick={startNewGame}
          >
            Play Again
          </button>
        </div>
      </div>
    );
  }

  // Render main game screen
  return (
    <div className="p-6 max-w-4xl mx-auto bg-gray-100 rounded-lg">
      <h1 className="text-2xl font-bold mb-4 text-center">Balatro-Inspired Game</h1>

      {/* Game stats */}
      <div className="flex justify-between mb-4">
        <div>
          <span className="font-bold">Chips:</span> {chips}
        </div>
        <div>
          <span className="font-bold">Ante:</span> {ante}
        </div>
        <div>
          <span className="font-bold">Round:</span> {round}
        </div>
        <div>
          <span className="font-bold">Deck:</span> {deck.length} cards
        </div>
        <div>
          <span className="font-bold">Hands Left:</span> {handsLeft}
        </div>
        <div>
          <span className="font-bold">Discards Left:</span> {discardCount}
        </div>
      </div>

      {/* Jokers */}
      <div className="mb-4">
        <h2 className="text-lg font-bold mb-2">Jokers:</h2>
        <div className="flex flex-wrap gap-2">
          {jokers.map((joker, index) => (
            <div key={index} className="p-2 bg-yellow-200 border border-yellow-500 rounded">
              <div>{joker.name}</div>
              <div>×{joker.multiplier}</div>
            </div>
          ))}
        </div>
      </div>

      {/* Last score */}
      {score > 0 && (
        <div className="mb-4 p-2 bg-green-100 border border-green-500 rounded">
          <h2 className="text-lg font-bold">Last Score: {score}</h2>
        </div>
      )}

      {/* Hand controls */}
      <div className="flex justify-between mb-4">
        <button
          className="px-4 py-2 flex items-center gap-2 bg-purple-500 text-white rounded hover:bg-purple-600"
          onClick={toggleSortType}
        >
          <ArrowUpDown size={16} />
          Sort by: {sortType === 'rank' ? 'Rank' : 'Suit'}
        </button>
        <div>
          <button
            className={`px-4 py-2 mr-2 rounded ${selectedCards.length > 0 && discardCount > 0 ? 'bg-orange-500 text-white hover:bg-orange-600' : 'bg-gray-300 text-gray-500 cursor-not-allowed'}`}
            onClick={discardSelected}
            disabled={selectedCards.length === 0 || discardCount <= 0}
          >
            Discard Selected ({selectedCards.length})
          </button>
          <button
            className={`px-4 py-2 rounded ${handsLeft > 0 ? 'bg-green-500 text-white hover:bg-green-600' : 'bg-gray-300 text-gray-500 cursor-not-allowed'}`}
            onClick={playHand}
            disabled={handsLeft <= 0}
          >
            Play Hand
          </button>
        </div>
      </div>

      {/* Cards */}
      <div className="flex flex-wrap gap-2 justify-center">
        {hand.map((card, index) => renderCard(card, index))}
      </div>
    </div>
  );
}